In [1]:
import pandas as pd
import random
from datetime import timedelta
from datetime import datetime

# This is a sample of what a product sheet would look like
# ID for vendor will mean something such as type of product or payment type
product_df = pd.DataFrame({
    "ProductID": [1,2,3,4],
    "Product": ["eggs","wine","cheese","candles"],
    "Vendor": [3001, 3002, 3003, 3004],
    "Unit_Price": [3.20, 16.20, 5.20,20.5],
    "Unit_Cost": [1.20, 3.40, 2.10, 10],
    "Initial_Inventory": [0,0,0,0],
    "Spread":[.30, .50, .10, .10],
    "PerInvoiceRng_Min":[10,10,10,10],
    "PerInvoiceRng_Max":[20,20,20,20]
})

# This would all come in as JSON not created in Python
Initial_Year = 2010
YearsIn_Operation = 10
first_year_rev = 10000000
trend = .05
modjulation = .01

x = first_year_rev
y = Initial_Year
coin = (1,2)

list_years = []



for year in range(YearsIn_Operation):
  Revenue = x
  Year = y
  Trended = Revenue * (1+trend)
  direction = random.sample(coin,1)
  if direction == 1:
    Modjulated = Trended * (1+modjulation)
  else:
    Modjulated = Trended * (1-modjulation)
  list_year_item = [Year, round(Revenue,2)]
  list_years.append(list_year_item)
  x = Modjulated
  y = y+1

# This is what the Data would look like in JSON
rev_df = pd.DataFrame(list_years,columns=["Year","Rev"])


In [36]:
from re import S
class PreProcessing:
  def __init__(self, product_df, year, Year_Rev):
    self.revenue = Year_Rev[1]
    self.products = product_df
    self.year = year

  def ProductProcessing(self):
    self.product_id_list = self.products['ProductID']
    self.vendor_list = self.products['Vendor']
    self.initial_inventory_list = self.products['Initial_Inventory']

    self.product_cost_info = self.products[['Product', 'Spread',
                                            'Unit_Price', 'Unit_Cost',
                                            'PerInvoiceRng_Min',
                                            'PerInvoiceRng_Max']].sort_values(by='Spread', ascending=False)
    product_cost = self.product_cost_info
    Rev = self.revenue
    yr_Product_Rev = []
    yr_Quantity_Sold = []
    Product_Cost = []
    Selling_Price = []
    yr_Product = []
    yr_noRecords_min = []
    yr_noRecords_max = []
    for product in self.product_cost_info['Product']:
      noRecords_min = product_cost[product_cost['Product']==product]['PerInvoiceRng_Min']
      noRecords_max = product_cost[product_cost['Product']==product]['PerInvoiceRng_Max']
      spread = product_cost[product_cost['Product']==product]['Spread'].item()
      unit_price = product_cost[product_cost['Product']==product]['Unit_Price'].item()
      unit_cost = product_cost[product_cost['Product']==product]['Unit_Cost'].item()
      price_sold = self.revenue * spread
      q_sold = round(price_sold / unit_price,0)
      p_sold = q_sold*unit_price
      if Rev >= p_sold:
        Rev = Rev - p_sold
      else:
        q_sold = round(Rev / unit_price,0)
        p_sold = q_sold*unit_price

      yr_Product.append(product)
      yr_Product_Rev.append(p_sold)
      yr_Quantity_Sold.append(q_sold)
      Product_Cost.append(unit_cost)
      Selling_Price.append(unit_price)
      yr_noRecords_min.append(noRecords_min)
      yr_noRecords_max.append(noRecords_max)

    processed_product_lists = [yr_Product, yr_Product_Rev,
                               yr_Quantity_Sold, Product_Cost, Selling_Price,
                               yr_noRecords_min, yr_noRecords_max]

    self.Processed_Product_df = pd.DataFrame(processed_product_lists).T
    self.Processed_Product_df.columns = ["yr_Product","yr_Product_Rev",
                                         "yr_Quantity_Sold","Product_Cost","Selling_Price",
                                         "yr_noRecordsMin","yr_noRecordsMax"]
    return [self.year, self.Processed_Product_df]


# Generate Sales Invoices
class CompanyGeneration:
  def __init__(self, year, product, yr_product_data):
    self.yr_product_data = yr_product_data

  def GenerateSales(self): # Already Per Product
    print(year, product)
    yr_Quantity_Sold = yr_product_data[yr_product_data['yr_Product']==product]['yr_Quantity_Sold'].item()
    Product_Cost = yr_product_data[yr_product_data['yr_Product']==product]['Product_Cost'].item()
    Selling_Price = yr_product_data[yr_product_data['yr_Product']==product]['Selling_Price'].item()
    MinRecords_PerInvoice = int(yr_product_data[yr_product_data['yr_Product']==product]['yr_noRecordsMin'])
    MaxRecords_PerInvoice = int(yr_product_data[yr_product_data['yr_Product']==product]['yr_noRecordsMax'])

    # Initial Quantity
    Quantity_Remaining = yr_Quantity_Sold
    Yr_Start = datetime(year, 1, 1)
    Yr_End = datetime(year, 12, 31)

    Sales_df_rows = []
    x = 0

    while Quantity_Remaining > 0:
      Order_Date = Yr_Start + timedelta(days=random.randint(0, (Yr_End-Yr_Start).days))
      # Product is Shipped 1-30 days after order
      Shipment_Date = Order_Date + timedelta(days = random.randint(1, 30))
      Records_PerInvoice = random.randint(MinRecords_PerInvoice, MaxRecords_PerInvoice)
      if Records_PerInvoice > Quantity_Remaining:
        Records_PerInvoice = Quantity_Remaining
      else:
        pass
      Rev = Records_PerInvoice * Selling_Price
      Tcost = Records_PerInvoice * Product_Cost
      
      Sales_df_rows.append([product, Records_PerInvoice, Order_Date, Shipment_Date, Selling_Price, Product_Cost, Rev, Tcost])
      Quantity_Remaining = Quantity_Remaining - Records_PerInvoice


    Sales_df = pd.DataFrame(Sales_df_rows, columns = ["Product", "Quantity", 
                                                      "Order_Date", "Shipment_Date", 
                                                      "Selling_Price", "Product_Cost", 
                                                      "Rev", "Total_cost"])
    print(Sales_df.info())

    # Return the Function and Call the Journal Entry Function


# Helper Functions
class RecordingFunctions:
  def __init__(self,General_Journal, Date, Amount, DebitAcc, CreditAcc):
    self.General_Journal = General_Journal
    self.Amount = Amount 
    self.Date = Date
    self.CreditAcc_list = CreditAcc
    self.DebitAcc_list = DebitAcc

  def JournalEntries(self):
    if isinstance(self.Amount, list) and isinstance(self.DebitAcc_list, list) and isinstance(self.CreditAcc_list, list):
      if len(self.Amount) == len(self.DebitAcc_list) + len(self.CreditAcc_list):
        len_ofDr = len(self.DebitAcc_list)
        DrAmounts = self.Amount[:len_ofDr]
        CrAmounts = self.Amount[len_ofDr:]
        Debits = zip(self.DebitAcc_list,DrAmounts)
        Credits = zip(self.CreditAcc_list,CrAmounts)
        for dr, amount in Debits: 
          Journal_Entry = [self.Date, dr, amount, "Debit"]
          self.General_Journal.loc[len(self.General_Journal)] = Journal_Entry
        for cr, amount in Credits:
          Journal_Entry = [self.Date, cr, amount, "Credit"]
          self.General_Journal.loc[len(self.General_Journal)] = Journal_Entry
      else: 
        print("ERROR Creating Journal Entries - Incorrect Number of Amounts")

    elif isinstance(self.Amount, int) or isinstance(self.Amount, float):
        Journal_Entry = [self.Date, self.DebitAcc_list, self.Amount, "Debit"]
        self.General_Journal.loc[len(self.General_Journal)] = Journal_Entry

        Journal_Entry = [self.Date, self.CreditAcc_list, self.Amount, "Credit"]
        self.General_Journal.loc[len(self.General_Journal)] = Journal_Entry

    else: 
      print("ERROR Creating Journal Entry- Incorrect DataTypes")
  # Input Accounts are Lists and it will handle any amount of debits or credits, You dont have to return anything




In [ ]:
Yr_Product_dict = {}

for yr in rev_df["Year"]:
  # Processing Products
  Year_Rev = rev_df[rev_df['Year']==yr].values.tolist()[0]
  generator_instance = PreProcessing(product_df, yr, Year_Rev)
  yr_product_data = generator_instance.ProductProcessing()[1]

  for product in yr_product_data['yr_Product'].unique():
    CompanyGeneration(yr, product, yr_product_data).GenerateSales()
    break


In [35]:
General_Journal = pd.DataFrame(columns = ["Date","Account","Amount","Balance"])



RecordingFunctions(General_Journal,"1/25/2023",[110,200,300,500],[1500],[2600,1200,1000]).JournalEntries()
